# Ray Tune -Ray Tune with Sklearn Hyperparameter Tuning

© 2019-2021, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)


# Tune's Scikit Learn Drop-in Replacements

<img src="https://docs.ray.io/en/latest/_images/tune-sklearn1.png" align="center" width="50%">

Scikit-Learn is one of the most widely used tools in the ML community for working with data, offering dozens of easy-to-use machine learning algorithms. However, to achieve high performance for these algorithms, you often need to perform **model selection**. Model selection is way to get the best performant model, after tuning over a set of parameters.

`tune-sklearn` is a module that integrates Ray Tune's hyperparameter tuning and scikit-learn's Classifier API. `tune-sklearn` has two APIs: [TuneSearchCV](https://docs.ray.io/en/latest/tune/api_docs/sklearn.html#tunesearchcv-docs), and [TuneGridSearchCV](https://docs.ray.io/en/latest/tune/api_docs/sklearn.html#tunesearchcv-docs). They are drop-in replacements for Scikit-learn's [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) and [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV), so you only need to change less than five lines in a standard Scikit-Learn script to use the API.

Let's compare Tune's Scikit-Learn APIs to the standard scikit-learn GridSearchCV. For this example, we'll be using `TuneGridSearchCV` with a stochastic gradient descent (SGD) [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html).

To start out, change the import statement to get tune-scikit-learn’s grid search cross validation interface.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Import Tune's replacements
from ray.tune.sklearn import TuneGridSearchCV
from ray.tune.sklearn import TuneSearchCV
from ray.tune.logger import LoggerCallback

# Other relevant imports
from sklearn.model_selection import train_test_split

# Use the stochastic gradient descent (SGD) classifier
from sklearn.linear_model import SGDClassifier

# import the classification dataset
from sklearn import datasets
from sklearn.datasets import make_classification
import numpy as np

Create classification data using `sklearn.datasets`

In [ ]:
def create_classification_data() -> (np.ndarray, np.ndarray):
    X, y = make_classification(
        n_samples=11000,
        n_features=1000,
        n_informative=50,
        n_redundant=0,
        n_classes=10,
        class_sep=2.5)
    return X, y

Create the classifcation data, training and test data sets, and define our hyperparameter
grid. 

In [ ]:
X, y = create_classification_data()
# Split the dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1000)

# Example parameters grid to tune from SGDClassifier
parameter_grid = {"alpha": [1e-4, 1e-1, 1], "epsilon": [0.01, 0.1]}

### Use Sklearn to train the model

In [ ]:
%%time
# n_jobs=-1 enables use of all cores does
sklearn_search = GridSearchCV(SGDClassifier(), parameter_grid, n_jobs=-1, verbose=True)
sklearn_search.fit(x_train, y_train)

In [ ]:
print("Best hyperparameters found were: ", sklearn_search.best_params_)

### Use Ray's Tune's drop-in replacement

And from there, we proceed just like how we would in Scikit-Learn’s interface!

The `SGDClassifier` has a `partial_fit` API, which enables it to stop fitting to the data for a certain hyperparameter configuration. If the estimator does not support early stopping, we would fall back to a parallel grid search.

As you can see, the setup here is exactly how you would do it for Scikit-Learn. Now, let's try fitting a model.



Start Ray on the local host

In [ ]:
import ray
ray.init(ignore_reinit_error=True)

Note the slight differences we introduced below:

 * a `early_stopping`, and
 * a specification of `max_iters` parameter

The ``early_stopping`` parameter allows us to terminate unpromising configurations. If ``early_stopping=True``, TuneGridSearchCV will default to using Tune's [ASHAScheduler](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-scheduler-hyperband). You can pass in a custom algorithm - see Tune's documentation on [schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers) for a full list to choose from.

``max_iters`` is the maximum number of iterations a given hyperparameter set could run for; it may run for fewer iterations if it is early stopped.

In [ ]:
%%time
tune_search = TuneGridSearchCV(
    SGDClassifier(), parameter_grid, early_stopping=True, 
    max_iters=10, name="AcademyTraining", verbose=1)
tune_search.fit(x_train, y_train)

In [ ]:
print("Best hyperparameters found were: ", tune_search.best_params)

## Using Bayesian Optimization

In addition to the grid search interface, tune-sklearn also provides an interface, `TuneSearchCV`, for sampling from **distributions of hyperparameters**.

In addition, you can easily enable Bayesian optimization over the distributions in only 2 lines of code:



In [ ]:
%%time
digits = datasets.load_digits()
x = digits.data
y = digits.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)

clf = SGDClassifier()
parameter_grid = {"alpha": (1e-4, 1), "epsilon": (0.01, 0.1)}

bayopt_tune_search = TuneSearchCV(
    clf,
    parameter_grid,
    search_optimization="bayesian",
    n_trials=3,
    early_stopping=True,
    max_iters=10,
    verbose=1,
)
bayopt_tune_search.fit(x_train, y_train)

In [ ]:
print("Best hyperparameters found were: ", bayopt_tune_search.best_params)

In [ ]:
ray.shutdown()